In [5]:
import pandas as pd
from pandas.io.json import json_normalize

import numpy as np

In [8]:
data = json_normalize(pd.read_json('./all.json').to_dict('records')).reset_index(drop=True)

In [10]:
# Filter out data
df = data[(data.rooms < 8) & (data['buildingData.city'] != 'Järvenpää')].copy()
df.loc[:, 'rooms'] = df.rooms.astype(int)
df.head()

,brand.id,brand.image,brand.name,buildingData.address,buildingData.buildingType,buildingData.city,buildingData.country,buildingData.district,buildingData.year,cardSubType,...,priceChanged,published,roomConfiguration,rooms,size,sizeLot,status,url,visits,visits_weekly
0,8411989.0,https://cdn.asunnot.oikotie.fi/R-nVPehXiLfCmHI...,Helsingin KOTI LKV Oy,Lähderanta 20 E,1.0,Espoo,None,Lähderanta,1965.0,[1],...,NaN,2018-09-14T17:40:35Z,"3h, keittiö, kph, parveke",3,66.0,115840.0,1,https://asunnot.oikotie.fi/myytavat-asunnot/es...,NaN,NaN
1,7630215.0,https://cdn.asunnot.oikotie.fi/uVeYeC5ZgqnVj8a...,"Kiinteistömaailma, Asuntoherttua Oy Viikki",Liusketie 6,1.0,Helsinki,None,Pihlajamäki,1963.0,[1],...,NaN,2018-09-14T17:31:40Z,"4h, k, kph, lasit.parveke",4,67.5,12407.1,1,https://asunnot.oikotie.fi/myytavat-asunnot/he...,NaN,NaN
2,13031499.0,None,Huoneistomaa Oy LKV,Kokkokalliontie 28-30,2.0,Vantaa,None,Kuninkaala Hakkilankallio,1990.0,[2],...,NaN,2018-09-14T17:29:59Z,3H+K+S+KPH+WC+P,3,68.0,3542.0,1,https://asunnot.oikotie.fi/myytavat-asunnot/va...,15.0,15.0
3,13031499.0,None,Huoneistomaa Oy LKV,Katiskatie 3,64.0,Helsinki,None,Ramsinniemi,2000.0,[64],...,NaN,2018-09-14T17:27:38Z,4H+K+S+PH+AULA+KHH+TKH+ASKH+2xWC+AT,5,163.0,2192.0,1,https://asunnot.oikotie.fi/myytavat-asunnot/he...,NaN,NaN
4,13629557.0,https://cdn.asunnot.oikotie.fi/fjTxrIrdoP_AUaz...,Blok,Miniatontie 8 A 3,2.0,Espoo,None,Soukanranta,1983.0,[2],...,NaN,2018-09-14T17:18:03Z,4h+k+kph+s+2erill.wc+khh+p+terassi+piha,4,104.5,NaN,1,https://asunnot.oikotie.fi/myytavat-asunnot/es...,NaN,NaN


In [13]:
df.loc[:, 'price'] = df.price.apply(lambda val: str(val).replace('\xa0', '').replace('€', '')).astype('float')

def norm_city(value):
    v = value.lower()
    if 'espoo' in v:
        return 'Espoo'
    elif 'helsinki' in v:
        return 'Helsinki'
    elif 'vantaa' in v or '01350' in v:
        return 'Vantaa'
    
    return value
    
df.loc[:, 'buildingData.city'] = df['buildingData.city'].apply(norm_city)
df.loc[:, 'buildingData.district'] = df['buildingData.district'].apply(lambda v: v.lower() if v else '')
df = df[(df['buildingData.district'] != '')]

df.loc[:, 'neighborhood'] = df.apply(lambda item: "{}, {}".format(item['buildingData.city'], item['buildingData.district']), axis=1)
df.loc[:, 'price_per_sq_m'] = df.apply(lambda item: item.price / item.size, axis=1)
df.loc[:, 'avg_price'] = df.apply(lambda item: item.price_per_sq_m * item.size, axis=1)

In [14]:
def percentile(n):
    def wrapper(x):
        return np.percentile(x, n)
    wrapper.__name__ = 'p%s' % n
    return wrapper

grouped = df.fillna(0).groupby(['neighborhood', 'rooms'])\
    .agg({'avg_price': ['mean']})\
    .astype(int)
grouped

avg_price
                                    mean
neighborhood             rooms          
Espoo, ala-soukka        3        188999
Espoo, bassenkylä        3         51589
Espoo, bemböle           4        358285
                         5        609000
                         6        880000
Espoo, bodom             4        349000
                         6        474000
                         7        640000
Espoo, brobacka          4        398000
Espoo, eestinlaakso      1        149704
                         3        278000
                         4        296000
Espoo, eestinmalmi       3        369999
                         4        419000
                         5        427934
Espoo, eestintaival      3         26502
Espoo, espoo             1        149902
                         2        211333
                         3        292666
                         4        406333
                         5        579000
Espoo, espoo lähderanta  4        229000
Espoo, espoo/kauniainen  5        898600
Espoo, espoon keskus     1        165289
                         2        198778
                         3        238525
                         4        331838
Espoo, espoonlahti       1        200000
                         2        203625
                         3        321433
...                                  ...
Vantaa, vantaanpuisto    1        109000
                         2        130637
                         3        151500
                         4        278000
Vantaa, vapaala          2        237333
                         3        220000
                         4        427500
                         5        378555
                         6        348000
Vantaa, varisto          2        139000
                         3        229200
                         4        254300
                         5        391000
                         6        625000
Vantaa, vestra           6        498000
Vantaa, vestra / keimola 3        328000
Vantaa, viertola         2        192500
                         4        273400
Vantaa, vierumäki        2        165000
                         3        211500
                         4        379000
                         5        362250
                         6        398333
Vantaa, viherkumpu       5        350000
Vantaa, voutila          4        459000
                         5        470000
Vantaa, ylästö           2         19197
                         3        245461
                         4        237874
                         5        443166

[1438 rows x 1 columns]

In [18]:
import json

def format_groups(groups):
    agg_key = ('avg_price', 'mean')
    items = list(grouped.to_dict()[agg_key].items())
    agg_stats = {}
    for item in items:
        (city_key, rooms), avg_price = item
        agg_stats.setdefault(city_key, [])
        agg_stats[city_key].append({'rooms': str(rooms), 'avg_price': avg_price})
    
    res = []
    for city_key, stats in agg_stats.items():
        city, district = city_key.split(',', 1)
        res.append({'city_key': city_key, 'stats': stats, 'city': city.strip(), 'district': district.strip()})
        res.append({'city_key': city_key, 'stats': stats,})
    return res

with open('./stats.json', 'w') as fd:
    json.dump(format_groups(grouped), fd, sort_keys=True, indent=2)